In [1]:
# Model
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
import torch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate
# Vector stores
import fitz  # PyMuPDF
import pdfplumber
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader, PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, KonlpyTextSplitter
from langchain_community.retrievers import BM25Retriever, KNNRetriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader, PDFPlumberLoader ,UnstructuredPDFLoader
from langchain_community.retrievers import BM25Retriever, KNNRetriever
from langchain.retrievers import EnsembleRetriever
from langchain_teddynote.retrievers import KiwiBM25Retriever, OktBM25Retriever
from langchain.docstore.document import Document
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import unicodedata
import pymupdf4llm
import time
import re
from konlpy.tag import Okt
from pdf2image import convert_from_path
import pytesseract
from konlpy.tag import Kkma
# etc
import os
import pandas as pd
from tqdm import tqdm
import unicodedata
import logging
from PyPDF2 import PdfReader
import json
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # GPU 사용 가능 여부 및 MPS 지원 여부 확인
print(device)

c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
from langchain_unstructured import UnstructuredLoader
# intfloat/multilingual-e5-small
# jhgan/ko-sroberta-multitask
def get_embedding():
    embeddings = HuggingFaceEmbeddings(
        model_name='jhgan/ko-sroberta-multitask',
        model_kwargs={'device': device},
        encode_kwargs={'normalize_embeddings': True})
    return embeddings

def normalize_string(s):
    try:
        normalized = unicodedata.normalize('NFC', s)
        return normalized.encode('utf-8', errors='replace').decode('utf-8')
    except Exception:
        return s
def clean_text(text):
    text = text.replace("�", " ").replace("", " ") # 잘못된 인코딩 문자 제거
    text = ' '.join(text.split())  # 여러 공백을 하나로 줄임
    return text


In [3]:
def get_docs(pdf_path):
    documents = []
    failed_pages = []  # 실패한 페이지를 추적하기 위한 리스트

    try:
        # 페이지 단위로 pymupdf4llm 사용
        md_read = pymupdf4llm.to_markdown(pdf_path, page_chunks=True)
        total_pages = len(md_read)

        for page_data in md_read:
            try:
                page_number = page_data.get('metadata', {}).get('page', None)
                if page_number is None:
                    raise ValueError("Page number missing in metadata")

                text = clean_text(normalize_string(page_data.get('text', '')))
                if not text:
                    raise ValueError("Empty text")  # 텍스트가 비어 있으면 예외 발생

                metadata = {
                    "file_path": pdf_path,
                    "page_number": page_number,
                    "total_pages": total_pages,
                    "tables": page_data.get('tables', [])
                }

                document = Document(page_content=text, metadata=metadata)
                documents.append(document)

            except Exception as e:
                logging.warning(f"Failed to process page {page_number} with pymupdf4llm: {e}")
                failed_pages.append(page_number)

    except Exception as e:
        logging.error(f"Failed to read PDF with pymupdf4llm: {e}")
        # pymupdf4llm 전체 실패 시 모든 페이지를 PyMuPDF로 처리하도록 설정
        failed_pages = list(range(1, len(fitz.open(pdf_path)) + 1))

    # 실패한 페이지에 대해 PyMuPDF로 재처리
    if failed_pages:
        try:
            doc = fitz.open(pdf_path)
            for page_num in failed_pages:
                page = doc.load_page(page_num - 1)  # 페이지 번호는 0부터 시작하기 때문에 -1
                text = page.get_text("text")
                if text:
                    text = clean_text(normalize_string(text))
                    metadata = {
                        "file_path": pdf_path,
                        "page_number": page_num,
                        "total_pages": doc.page_count
                    }
                    document = Document(page_content=text, metadata=metadata)
                    documents.append(document)

        except Exception as e:
            logging.error(f"Failed to read PDF with PyMuPDF: {e}")

    return documents

def chunk_documents(docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=60,
        separators=["\n\n", ".", "?", "!", "\n", "\t"]
    )
    chunks = []

    for doc in docs:
        text = doc.page_content
        metadata = doc.metadata

        # 표가 있는 경우 페이지 전체를 하나의 청크로 사용
        if metadata.get('tables'):
            chunks.append(Document(page_content=text, metadata=metadata))
        else:
            text_chunks = text_splitter.split_text(text)
            chunks.extend([Document(page_content=chunk, metadata=metadata) for chunk in text_chunks])

    return chunks

def make_db(df):
    documents = []
    pdf_files = df['Source_path'].unique()

    # tqdm으로 파일 처리 진행 상황 표시
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(get_docs, pdf_files), total=len(pdf_files), desc="Processing PDFs"))

    for result in results:
        documents.extend(result)

    # 정규화
    for doc in documents:
        doc.page_content = normalize_string(doc.page_content)

    # 표가 망가지지 않도록 청크 분할 처리
    chunks = chunk_documents(documents)
    print(f"Total number of chunks: {len(chunks)}")

    faiss = FAISS.from_documents(chunks, embedding=get_embedding())
    bm = OktBM25Retriever.from_documents(chunks)

    return faiss, bm

In [4]:
def fewshot_db(df):
    df = df.drop('SAMPLE_ID', axis=1)
    df = df.drop('Source_path', axis=1)
    df = df.to_dict(orient='records')
    print("Loaded Fewshot Set:", len(df))
    to_vectorize = ["\n\n".join(normalize_string(value) for value in example.values()) for example in df]
    faiss = FAISS.from_texts(to_vectorize, embedding=get_embedding())
    # bm = BM25Retriever.from_texts(to_vectorize)
    # knn = KNNRetriever.from_texts(to_vectorize, embeddings=get_embedding())
    return faiss

In [5]:
train_df = pd.read_csv('train.csv', encoding='utf-8')
test_df = pd.read_csv('test.csv', encoding='utf-8')

In [6]:
def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for i, doc in enumerate(docs):
        #context += f"Document {i+1}\n"
        doc.page_content = doc.page_content.replace("{", "(")
        doc.page_content = doc.page_content.replace("}", ")")
        
        context += doc.page_content
        context += '\n\n'
    return context.strip()

In [7]:
def setup_llm_pipeline(model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"):
    # 토크나이저 로드 및 설정
        # 양자화 설정 적용
    bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_use_double_quant=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.bfloat16
    )
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config,low_cpu_mem_usage=True)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        return_full_text=False,

        max_new_tokens=256,
        eos_token_id = terminators,
        pad_token_id = tokenizer.eos_token_id
    )

    llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return llm
llm = setup_llm_pipeline()

Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.35s/it]


In [8]:
from collections import Counter
def calculate_f1_score(true_sentence, predicted_sentence, sum_mode=True):

    #공백 제거
    true_sentence = ''.join(true_sentence.split())
    predicted_sentence = ''.join(predicted_sentence.split())
    
    true_counter = Counter(true_sentence)
    predicted_counter = Counter(predicted_sentence)

    #문자가 등장한 개수도 고려
    if sum_mode:
        true_positive = sum((true_counter & predicted_counter).values())
        predicted_positive = sum(predicted_counter.values())
        actual_positive = sum(true_counter.values())

    #문자 자체가 있는 것에 focus를 맞춤
    else:
        true_positive = len((true_counter & predicted_counter).values())
        predicted_positive = len(predicted_counter.values())
        actual_positive = len(true_counter.values())

    #f1 score 계산
    precision = true_positive / predicted_positive if predicted_positive > 0 else 0
    recall = true_positive / actual_positive if actual_positive > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1_score

def calculate_average_f1_score(true_sentences, predicted_sentences):
    
    total_precision = 0
    total_recall = 0
    total_f1_score = 0
    
    for true_sentence, predicted_sentence in zip(true_sentences, predicted_sentences):
        precision, recall, f1_score = calculate_f1_score(true_sentence, predicted_sentence)
        total_precision += precision
        total_recall += recall
        total_f1_score += f1_score
    
    avg_precision = total_precision / len(true_sentences)
    avg_recall = total_recall / len(true_sentences)
    avg_f1_score = total_f1_score / len(true_sentences)
    
    return {
        'average_precision': avg_precision,
        'average_recall': avg_recall,
        'average_f1_score': avg_f1_score
    }

In [9]:
def extract_answer(response):
    # AI: 로 시작하는 줄을 찾아 그 이후의 텍스트만 추출
    lines = response.split('\n')
    for line in lines:
        line = line.replace('**', '')
        if line.startswith('Answer:'):
            return line.replace('Answer:', '').strip()
        if line.startswith('assistant:'):
            return line.replace('assistant:', '').strip()
    return response.strip()  # AI: 를 찾지 못한 경우 전체 응답을 정리해서 반환



def run (test,dataset,llm,verbose=False):
    results = []
    for i, row in (dataset.iterrows()):
# 
        full_template = "<|begin_of_text|>"
        full_template += """<|start_header_id|>system<|end_header_id|>
당신은 유용한 금융 정보 QnA 챗봇입니다.
문맥 정보들을 바탕으로 질문에 답변하세요.
객관적이고 공식적인 문체를 사용하세요.
어떤 서론이나 배경 설명 없이 바로 핵심 내용을 전달해주세요.
<|eot_id|>
"""
        question = row['Question']          
        full_template += """ """
        contexts = test.invoke(normalize_string(question))
        contexts = format_docs(contexts)
        full_template += """<|start_header_id|>user<|end_header_id|>Question: {input} \n1 문장으로 작성하세요.\n\n"""
        full_template += f"""Context: {contexts}<|eot_id|>\n\n"""
        full_template += """<|start_header_id|>assistant<|end_header_id>"""
        
        prompt = PromptTemplate(template=full_template, input_variables=["input"])
        qa_chain = (
        {
            "input": RunnablePassthrough(),
        }
        | prompt
        | llm
        | StrOutputParser()
        )

        answer = qa_chain.invoke(input=question)
        #answer = extract_answer(answer)
        results.append({
            "Question": question,
            "Answer": answer,
            "Source": row['Source']
        })
        if verbose:
            print(f"{i}/{len(dataset)}")
            print("Question: ", question, end=" | ")
            print("Context Number |",len(contexts))
            print("Answer: ", results[-1]['Answer'])
            try:
                print(calculate_f1_score(row['Answer'],answer)[2]," | ","REAL Answer: ",row['Answer'])
            except:
                pass
    return results

In [10]:
from sklearn.model_selection import KFold
import copy

weight = [0.5,0.5]
train_faiss_db, train_bm_retrievier = make_db(train_df) 

train_k = 2
train_bm_retrievier.k = train_k
#knn_retriever.k = train_k
train_faiss_retriever = train_faiss_db.as_retriever(search_type="mmr",search_kwargs={'k':train_k} )
train_ensemble_retriever = EnsembleRetriever(
    retrievers=[train_bm_retrievier, train_faiss_retriever], weights=weight
)


fewshot_k = 3

k_folds = 4
fold_results = []
kf = KFold(n_splits=k_folds, shuffle=True, random_state=52)
for fold, (train_index, val_index) in enumerate(kf.split(train_df)):
    fold_result = []
    train_set = train_df.iloc[train_index]
    val_set = train_df.iloc[val_index]

    pred = run(train_ensemble_retriever, val_set, llm, verbose=True)
    result = pd.DataFrame()
    result['pred'] = [result['Answer'] for result in pred]
    val_set.index = range(len(val_set))
    result['gt'] = val_set['Answer']
        
    result = calculate_average_f1_score(result['gt'], result['pred'])
    print(result)
    fold_results.append(result)
    break

ERROR:root:Failed to read PDF with pymupdf4llm: not a textpage of this page
ERROR:root:Failed to read PDF with pymupdf4llm: not a textpage of this page
ERROR:root:Failed to read PDF with pymupdf4llm: not a textpage of this page
ERROR:root:Failed to read PDF with pymupdf4llm: not a textpage of this page
ERROR:root:Failed to read PDF with pymupdf4llm: not a textpage of this page
ERROR:root:Failed to read PDF with pymupdf4llm: not a textpage of this page
Processing PDFs:   0%|          | 0/16 [00:00<?, ?it/s]ERROR:root:Failed to read PDF with pymupdf4llm: not a textpage of this page
ERROR:root:Failed to read PDF with pymupdf4llm: not a textpage of this page
ERROR:root:Failed to read PDF with pymupdf4llm: not a textpage of this page
ERROR:root:Failed to read PDF with pymupdf4llm: not a textpage of this page
ERROR:root:Failed to read PDF with pymupdf4llm: not a textpage of this page
ERROR:root:Failed to read PDF with pymupdf4llm: not a textpage of this page
ERROR:root:Failed to read PDF wit

Total number of chunks: 1562


c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\models\llama\modeling_llama.py:660: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


1/124
Question:  2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요? | Context Number | 1459
Answer:  1. 2024년 중앙정부의 예산 지출은 656.6조원으로 구성되어 있으며, 일반회계가 356.5조원으로 중앙정부 총지출의 54.3%를 차지합니다.
0.6890756302521008  |  REAL Answer:  2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7조원으로 구성되어 있습니다.
4/124
Question:  2024년 총수입은 얼마이며, 예산수입과 기금수입은 각각 몇 조원인가요? | Context Number | 1622
Answer:  2024년도 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 216.7조원입니다.
0.9896907216494846  |  REAL Answer:  2024년 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 216.7조원입니다.
6/124
Question:  2024년의 기금수입은 어떻게 구성되어 있나요? | Context Number | 1555
Answer:  1. 총수입은 일반회계수입 + 특별회계수입 + 기금수입 - 내부거래 - 보전거래입니다.
0.2947368421052632  |  REAL Answer:  2024년도 기금수입은 사회보장성기금 92.3조원, 경상이전수입 39.6조원, 기타 84.7조원으로 구성되어 있습니다.
11/124
Question:  2024년 총지출 기준 예산의 일반회계와 특별회계의 비중이 각각 얼마인가? | Context Number | 2362
Answer:  2024년 총지출 기준 예산의 일반회계와 특별회계의 비중은 각각 71.6%와 28.4입니다.
0.8  |  REAL Answer:  2024년 총지출 중 일반회계와 특별회계의 비중은 각각 54.3%와 12.4%이다.


In [ ]:
# from save_module import save


# weight = [0.5,0.5]
# test_faiss_db, test_bm_retrievier = make_db(test_df)

# test_k = 2
# test_bm_retrievier.k = test_k
# #test_knn_retriever.k = test_k
# test_faiss_retriever = test_faiss_db.as_retriever(search_type="mmr",search_kwargs={'k':test_k} )
# test_ensemble_retriever = EnsembleRetriever(
#     retrievers=[test_bm_retrievier, test_faiss_retriever], weights=weight
# )


# results = run(test_ensemble_retriever, test_df, llm, verbose=True)
# save(results)